# pymatgen library

For more information and usage, see here: https://pymatgen.org

# Do not install like this if you do not want to currupt your other Anaconda packages

Install using:

pip install pymatgen

or

conda install pymatgen

If ruamel.yaml is needed, further install

pip install ruamel.yaml

or

conda install ruamel.yaml

# Install like this. Create a new environment for such special packages

### Make sure your conda is updated
conda update conda
### Create a new environment for pymatgen. I have named this environment as pymatgen. You can choose any name.
conda create -n pymatgen
### We want to install pymatgen inside pymagen environment. So, activate the environment.
conda activate pymatgen
### Install pymatgen package inside pymatgen environment.
conda install --channel conda-forge pymatgen
### Most likely Jupyter notebook is not installed inside pymatgen environment. Install Jupyter notebook.
conda install --channel conda-forge jupyter

In [1]:
#import pymatgen as pymat
from pymatgen.core.periodic_table import Element

In [2]:
#si = mg.Element("Si")
E_Si=Element("Si").youngs_modulus

In [3]:
E_Si

47.0

In [4]:
print(Element("Si").melting_point)

1687.0 K


### Getting a more detailed dataset
The following is based on nanohub.org portal. In this section we will query both [Pymatgen](http://pymatgen.org/) and [Mendeleev](https://mendeleev.readthedocs.io/en/stable/) to get a complete set of properties per element. We will use this data to create the cases from which the model will train and test.
<br>
<br>
In this first snippet of code we will import all relevant libraries, the elements that will be turned into cases and the properties that will serve as the attributes for the cases. We will get 47 entries (which is a small dataset), but should give us a somewhat accurate prediction. It is important to note that more entries would move the prediction closer to the real value, and so would more attributes.
<br>
<br>
The elements listed were chosen because querying them for these properties yields a dataset with no unknown values, and because they represent the three most common crystallographic structures.

In [3]:
import pymatgen.core as pymat
import mendeleev as mendel
import random

In [13]:
fcc_elements = ["Ag", "Al", "Au", "Cu", "Ir", "Ni", "Pb", "Pd", "Pt", "Rh", "Th", "Yb"]
bcc_elements = ["Ba", "Ca", "Cr", "Cs", "Eu", "Fe", "Li", "Mn", "Mo", "Na", "Nb", "Rb", "Ta", "V", "W" ]
hcp_elements = ["Be", "Cd", "Co", "Dy", "Er", "Gd", "Hf", "Ho", "Lu", "Mg", "Re", 
                "Ru", "Sc", "Tb", "Ti", "Tl", "Tm", "Y", "Zn", "Zr"]

elements = fcc_elements + bcc_elements + hcp_elements

# Shuffle the list of element
#random.Random(1).shuffle(elements)
random.shuffle(elements)

#Note that different properties have been added from the two databases.
querable_mendeleev = ["atomic_number", "atomic_volume", "boiling_point", "en_ghosh",  "evaporation_heat", "heat_of_formation",
                     "lattice_constant", "melting_point", "specific_heat"]
querable_pymatgen = ["atomic_mass", "atomic_radius", "electrical_resistivity","molar_volume", "bulk_modulus", "youngs_modulus",
                     "average_ionic_radius", "density_of_solid", "coefficient_of_linear_thermal_expansion"]
querable_values = querable_mendeleev + querable_pymatgen

In [15]:
print(elements)

['Gd', 'Ni', 'Cu', 'Y', 'Rh', 'Tl', 'Mn', 'Au', 'Be', 'Co', 'Zn', 'Tb', 'Lu', 'Fe', 'Yb', 'Cd', 'V', 'W', 'Na', 'Pb', 'Sc', 'Ag', 'Ti', 'Zr', 'Hf', 'Nb', 'Pt', 'Re', 'Dy', 'Th', 'Tm', 'Ru', 'Al', 'Ba', 'Mg', 'Ca', 'Pd', 'Ho', 'Mo', 'Cs', 'Ir', 'Eu', 'Li', 'Cr', 'Rb', 'Er', 'Ta']


### As before, we will use the database queries to populate lists which can be displayed by the [Pandas](https://pandas.pydata.org/) library in a user-friendly table with the properties as the column headers.

In [16]:
import pandas as pd

In [17]:
all_values = [] # Values for Attributes
all_labels = [] # Crystal structure labels (0 = fcc, 1 = bcc, 2 = hcp)

for item in elements:
    element_values = []
    
    # This section queries Mendeleev
    element_object = mendel.element(item)
    for i in querable_mendeleev:    
        element_values.append(getattr(element_object,i))

    # This section queries Pymatgen
    element_object = pymat.Element(item)    
    for i in querable_pymatgen:
        element_values.append(getattr(element_object,i))
        
    all_values.append(element_values) # All lists are appended to another list, creating a List of Lists
    
    if (item in fcc_elements):
        all_labels.append([1, 0, 0]) # The crystal structure labels are assigned here
    elif (item in bcc_elements):
        all_labels.append([0, 1, 0]) # The crystal structure labels are assigned here
    elif (item in hcp_elements):
        all_labels.append([0, 0, 1]) # The crystal structure labels are assigned here

#print("What is inside element_values")
#print(element_values)
#print("What is inside all_labels")
#print(all_labels)

# Pandas Dataframe
df = pd.DataFrame(all_values, columns=querable_values)

# We will patch some of the values that are not available in the datasets.

# Value for the CTE of Cesium
index_Cs = df.index[df['atomic_number'] == 55]
df.iloc[index_Cs, df.columns.get_loc("coefficient_of_linear_thermal_expansion")] = 0.000097 
# Value from: David R. Lide (ed), CRC Handbook of Chemistry and Physics, 84th Edition. CRC Press. Boca Raton, Florida, 2003

# Value for the CTE of Rubidium
index_Rb = df.index[df['atomic_number'] == 37]
df.iloc[index_Rb, df.columns.get_loc("coefficient_of_linear_thermal_expansion")] = 0.000090 
# Value from: https://www.azom.com/article.aspx?ArticleID=1834

# Value for the Evaporation Heat of Ruthenium
index_Ru = df.index[df['atomic_number'] == 44]
df.iloc[index_Ru, df.columns.get_loc("evaporation_heat")] = 595 # kJ/mol 
# Value from: https://www.webelements.com/ruthenium/thermochemistry.html

# Value for the Bulk Modulus of Zirconium
index_Zr = df.index[df['atomic_number'] == 40]
df.iloc[index_Zr, df.columns.get_loc("bulk_modulus")] = 94 # GPa 
# Value from: https://materialsproject.org/materials/mp-131/

df.head

<bound method NDFrame.head of     atomic_number  atomic_volume  boiling_point  en_ghosh  evaporation_heat  \
0              64          19.90        3539.00  0.194400             398.0   
1              28           6.60        3005.00  0.147207             378.6   
2              29           7.10        2840.00  0.151172             304.6   
3              39          19.80        3611.00  0.121699             367.0   
4              45           8.30        4000.00  0.140838             494.0   
5              81          17.20        1730.00  0.173447             162.4   
6              25           7.39        2235.00  0.135284             221.0   
7              79          10.20        3080.00  0.261370             340.0   
8               4           5.00        3243.00  0.144986             309.0   
9              27           6.70        3143.00  0.143236             389.1   
10             30           9.20        1180.00  0.155152             114.8   
11             65     